# ATP Data Exploration

In [18]:
import numpy as np
import pandas as pd
from siuba import *

import altair as alt
import altair_saver
from plotnine import *

from IPython.display import Markdown, HTML, Image
import ipywidgets as widgets
from ipywidgets import interact, interactive

from calitp import to_snakecase
import intake

from shared_utils import altair_utils
from shared_utils import styleguide

from dla_utils import _dla_utils

In [5]:
main_details = to_snakecase(pd.read_excel("gs://calitp-analytics-data/data-analyses/dla/atp/Main Details.xls"))
project_details = to_snakecase(pd.read_excel("gs://calitp-analytics-data/data-analyses/dla/atp/Project Details.xls"))

In [9]:
main_details.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 882 entries, 0 to 881
Data columns (total 83 columns):
 #   Column                        Non-Null Count  Dtype         
---  ------                        --------------  -----         
 0   a1_imp_agcy_city              882 non-null    object        
 1   a1_imp_agcy_contact           882 non-null    object        
 2   a1_imp_agcy_email             882 non-null    object        
 3   a1_imp_agcy_fed_ma_num        742 non-null    object        
 4   a1_imp_agcy_ma                882 non-null    object        
 5   a1_imp_agcy_name              882 non-null    object        
 6   a1_imp_agcy_phone             882 non-null    object        
 7   a1_imp_agcy_state_ma_num      780 non-null    object        
 8   a1_imp_agcy_street            882 non-null    object        
 9   a1_imp_agcy_title             882 non-null    object        
 10  a1_imp_agcy_zip               882 non-null    int64         
 11  a1_letter_of_intent           87

In [11]:
project_details.sample()

,b_sig_inter_new_bike_boxes,b_class_1,b_class_2,b_class_3,b_class_4,a4_bike_gap_pct,b_light_intersection,b_mid_block_new_rrfb_signal,b_mid_block_surf_improv,b_bsp_new_bikes,...,a4_collab_other,a4_colab_other_descr,a4_plan_ped,a4_plan_bike,a4_plan_atp,a4_plan_school_routes,a4_row_open_street_demo,project_app_id,project_cycle,awarded
422,0,0,0,0,18800,NaN,0,0,0,0,...,N,NaN,N,N,N,N,No,"10-Stockton, City of-3",CYCLE 5,N


In [13]:
project_details.project_cycle.value_counts()

CYCLE 5    448
CYCLE 6    434
Name: project_cycle, dtype: int64

In [16]:
project_details>>count(_.project_app_id)>>filter(_.n>1)

,project_app_id,n
4,"1-Eureka, City of-1",2
5,"1-Eureka, City of-2",2
9,1-Lake County-1,2
10,1-Mendocino Council of Governments-1,2
16,10-Calaveras County-1,2
...,...,...
601,"8-Wildomar, City of-2",2
602,"8-Yucaipa, City of-1",2
603,"9-Bishop, City of-1",2
605,"9-Tehachapi, City of-1",2


In [15]:
project_details>>group_by(_.project_cycle)>>count(_.project_app_id)>>filter(_.n>1)

,project_cycle,project_app_id,n


* multiple project ids, but no duplicates by project cycle. 

## Merging Data

In [19]:
df = pd.merge(main_details, project_details, how="outer", on=["project_app_id", "project_cycle"], indicator='matches')

In [24]:
(df>>filter(_.project_app_id =='1-Mendocino Council of Governments-1')>>select(_.project_app_id, 
                                                                              _.project_cycle,
                                                                             _.matches,
                                                                              _.agency_app_num))

,project_app_id,project_cycle,matches,agency_app_num
405,1-Mendocino Council of Governments-1,CYCLE 5,both,1
750,1-Mendocino Council of Governments-1,CYCLE 6,both,1
